In [1]:
import pandas as pd
import numpy as np
import csv
import random
import datetime

Читаем датасет

In [30]:
FULL_PATH = r""

df_u = pd.read_csv(FULL_PATH + "users.csv", sep=';',error_bad_lines=False, quoting=csv.QUOTE_NONE, encoding='utf-8', decimal=',', quotechar='"')
df_t = pd.read_csv(FULL_PATH + "transactions.csv", sep=';',error_bad_lines=False, quoting=csv.QUOTE_NONE, encoding='utf-8', decimal=',', quotechar='"')

In [31]:
df_t['SUM_TRANS'] = df_t['SUM_TRANS'].astype(float)

In [32]:
df_u.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19995 entries, 0 to 19994
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               19995 non-null  int64  
 1   MM_IN_BANK       19995 non-null  int64  
 2   MM_W_CARD        19995 non-null  int64  
 3   AGE              19995 non-null  int64  
 4   GENDER           19995 non-null  int64  
 5   EDUCATION_LEVEL  19995 non-null  int64  
 6   MARITAL_STATUS   19995 non-null  int64  
 7   DEPENDANT_CNT    19995 non-null  int64  
 8   INCOME_MAIN_AMT  10516 non-null  object 
 9   REG_CODE         19995 non-null  int64  
 10  COUNT_TRANS      18713 non-null  float64
dtypes: float64(1), int64(9), object(1)
memory usage: 1.7+ MB


In [33]:
def csv_locale_index(pd1):
    pd1 = pd1.dropna()
    
    all_tovar = []
    for row in pd1.values:
        all_tovar.append(row[5].split("\\")[0].strip())
    all_tovar = list(set(all_tovar))
    dickMarka = {all_tovar[i]: i for i in range(len(all_tovar))}
    
    location_name_index_list = []
    location_names_list = []
    pd_values = pd1.values
    for row_index in range(len(pd_values)):
        value = pd_values[row_index][5].split("\\")[0].strip()
        index = dickMarka[value]
#         print(value, pd1.iloc[row_index]['LOCATION_NAME'])
#         pd1.iloc[row_index]['LOCATION_NAME'] = value
        location_name_index_list.append(index)
        location_names_list.append(value)
        
    pd1['LOCATION_NAME_INDEX'] = location_name_index_list
    pd1['LOCATION_NAME'] = location_names_list
#     pd1.to_csv(r'C:\Users\Артур\MarkBestGay/vlad.csv', index=False, sep=";")
    print("Worked!")
    print(pd1)
    
    return (pd1, dickMarka)

Читем продукты(магазины), которые будем рекомендовать

In [34]:
(df_t, label_dict) = csv_locale_index(df_t)
df_t.head()

<ipython-input-33-91c18f7186ea>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd1['LOCATION_NAME_INDEX'] = location_name_index_list
<ipython-input-33-91c18f7186ea>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd1['LOCATION_NAME'] = location_names_list


Worked!
                         ID  PROD_TYPE           TRANS_DTTM  MCC_CODE  \
0        500000000048605476          1  2021-07-26 00:00:00      5912   
1        500000000627114675          2  2021-10-25 13:11:54      5814   
19       500000002402742235          2  2021-09-30 19:55:00      5813   
25       500000000001441130          1  2021-08-15 07:48:40      5941   
29       500000000598956263          2  2021-10-30 02:16:39      5812   
...                     ...        ...                  ...       ...   
1284494  500000000350335362          2  2021-10-26 13:58:57      5200   
1284495  500000000889236204          1  2021-10-21 09:33:29      5411   
1284501  500000000049785794          1  2021-10-31 15:27:47      5411   
1284502  500000000128977696          1  2021-07-09 10:09:48      5732   
1284504  500000000048796429          1  2021-10-29 04:35:10      5999   

         SUM_TRANS               LOCATION_NAME  LOCATION_NAME_INDEX  
0          2451.00  APTEKA ZDOROV.RU>MOSCOW R

,ID,PROD_TYPE,TRANS_DTTM,MCC_CODE,SUM_TRANS,LOCATION_NAME,LOCATION_NAME_INDEX
0,500000000048605476,1,2021-07-26 00:00:00,5912,2451.0,APTEKA ZDOROV.RU>MOSCOW RU,42981
1,500000000627114675,2,2021-10-25 13:11:54,5814,350.0,YANDEX EDA,49268
19,500000002402742235,2,2021-09-30 19:55:00,5813,350.0,PAB PATRIK,85279
25,500000000001441130,1,2021-08-15 07:48:40,5941,3133.0,5118 SM DVF,76177
29,500000000598956263,2,2021-10-30 02:16:39,5812,1300.0,STOLOVAYA,41749


Проходимся циклом, по каждому пользователю и делаем предсказание:

Если у нас есть только 5-10 транзакций, этого мало, поэтому основываясь на кластере пользователя добавляем популярные в этом кластере покупки (они через какое-то время исчезнуть, но надо же в начале что-то рекомендовать)


Если у нас есть большое количество транзакций применяем ALS, который добавляет немного покупок по кластерам и случайных покупок. Так же какую то часть предсказаний мы предлагаем пользователю то, чего он ещё не покупал, но другие похожие(по покупкам) пользователи покупали

In [35]:
uniq_ids_u = df_u['ID'].unique()
uniq_ids_t = df_t['ID'].unique()
print(len(uniq_ids_u))
print(len(uniq_ids_t))

19995
129540


In [36]:
def create_not_real_clusters(n):
    clusters = []
    for i in range(n):
        if(i % 2 == 0):
            clusters.append(1)
        else:
            clusters.append(2)
    return clusters

In [37]:
df_u['CLUSTER'] = create_not_real_clusters(df_u.shape[0])

Находим топ самых продаваемых товаров по кластеру

In [38]:
def find_top_dict(user):
    c_users = df_u[df_u['CLUSTER'] == user['CLUSTER'].iloc[0]]
    return df_t[df_t['ID'].isin(c_users['ID'])]['LOCATION_NAME'].value_counts()[:10]

Находим среднюю сумму по кластеру и товару

In [43]:
def mean_sum_by_cluster_location(cluster, location):
    c_users = df_u[df_u['CLUSTER'] == cluster]
    trans = df_t[df_t['ID'].isin(c_users['ID'])]
    trans = trans[trans['LOCATION_NAME_INDEX'] == location]
    return trans['SUM_TRANS'].mean()

Если пользователь новый, то добавляем транзакции

In [40]:
def add_transactions(df_t, user):
    top = find_top_dict(user)
    cluster = user['CLUSTER'].iloc[0]
    for i in range(0, 15):
        index = random.randint(0, 9)
        l_index = label_dict[top.index[index]]
        ser = pd.Series({
            'ID' : user['ID'],
            'PROD_TYPE': 1,
            'MCC_CODE' : -1, #такое значение MCC чтобы потом удалить сгенерированные транзакции,
            'TRANS_DTTM' : datetime.datetime.now(),
            'SUM_TRANS' : mean_sum_by_cluster_location(cluster, l_index),
            'LOCATION_NAME' : top[index],
            'LOCATION_NAME_INDEX' : l_index
        })
        df_t.append(ser, ignore_index=True)
    return df_t

In [45]:
def make_sparce_matrix(pd, locale_encoder, user): # датафрейм, словарик назваие-цифры, пользователь
    user = df_u[df_u['ID'] == user]
    idx = []
    values = []
    items = [] 
    top = []
    user_pd = pd[pd['ID'] == user]
    for trans_index in range(len(user_pd)):
        trans = user_pd.iloc[trans_index]
        items.append(trans['LOCATION_NAME'])
        
    if(user['CHANGE'].iloc[0] == 1):
        top = find_top_dict(user)
        for i in range(3):
            index = random.randint(0, 9)
            items.append(label_dict[top.index[index]])
    
    n_items = len(items)
    

    for pid in items:
        value = 1.0
        idx.append(locale_encoder[pid])
        if(items in top):
            value = value * 1.3
        
        values.append(value)

    return sp.coo_matrix(
        (np.array(values).astype(np.float32), ([0] * len(idx), idx)), shape=(1, len(locale_encoder)),
    )
        

In [44]:
from tqdm import tqdm
for id_u in tqdm(uniq_ids_u):
    user = df_u[df_u['ID'] == id_u]
    trans = df_t[df_t['ID'] == id_u]
    if(user['COUNT_TRANS'].iloc[0] < 5):
        df_t = add_transactions(df_t, user)
    else:
        df_u.loc[df_u['ID'] == id_u, 'CHANGE'] = 1

  3%|██▉                                                                                                       | 556/19995 [09:09<5:20:00,  1.01it/s]


KeyboardInterrupt: 

In [ ]:
rows = []
unique_users = df_t['ID'].unique()
train_users = unique_users[:-100]
test_users = unique_users[-100:]

for user_index in tqdm(range(len(train_users))):
    rows.append(make_sparce_matrix(df_test, label_dict, train_users[user_index]))

In [ ]:
x_sparse = sp.vstack(rows).tocsr()
x_sparse.shape

In [ ]:
model = implicit.als.AlternatingLeastSquares(factors=16, regularization=0.0, iterations=8)
# model = implicit.nearest_neighbours.CosineRecommender(K=200)
model.fit(x_sparse.T)

In [46]:
locale_encoder_reverse = {}
for k, v in tqdm(locale_encoder.items()):
    locale_encoder_reverse[v] = k
# inv_map = {v: k for k, v in my_map.items()}

NameError: name 'locale_encoder' is not defined

In [ ]:
def get_user_transactions(user, pd):
    user_pd = pd[pd['ID'] == user]
    return list(user_pd['LOCATION_NAME_INDEX'])

In [ ]:
get_user_transactions(train_users[0], location_MarkDick)

In [ ]:
def average_precision(actual, recommended, k=30):
    ap_sum = 0
    hits = 0
    for i in range(k):
        product_id = recommended[i] if i < len(recommended) else None
        if product_id is not None and product_id in actual:
            hits += 1
            ap_sum += hits / (i + 1)
    return ap_sum / k

def normalized_average_precision(actual, recommended, k=30):
    actual = set(actual)
    if len(actual) == 0:
        return 0.0

    ap = average_precision(actual, recommended, k=k)
    ap_ideal = average_precision(actual, list(actual)[:k], k=k)
    return ap / ap_ideal

In [ ]:
m_ap = []

for user in test_users:
    row_sparse = make_sparce_matrix(location_MarkDick, locale_encoder, user)
    raw_recs = model.recommend(0, row_sparse, N=30, filter_already_liked_items=False, recalculate_user=True)
    recommended_items = [rec[0] for rec in raw_recs]
#     print(raw_recs, recommended_items)
    
    gt_items = get_user_transactions(user, location_MarkDick)
    m_ap.append(normalized_average_precision(gt_items, recommended_items, k=30))
    
print(np.mean(m_ap))